In [ ]:
# !wget -nc https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos/pub/FEAGA_FEADER_2014.zip]
# !wget -nc https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos/pub/FEAGA_FEADER_2015.zip

In [ ]:
!wget -nc https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos/pub/FEAGA_FEADER_2016.zip

In [1]:
import pandas as pd
import numpy as np

In [2]:
YEAR = 2016

In [7]:
df = pd.read_csv('FEAGA_FEADER_%s.zip' % YEAR, delimiter=';', encoding='latin1', skiprows=1, compression='zip')
df = df[~(df['medida'] == 'Total')]
df.head(6)

,exercicio,denominacao,distrito,descricao_distrito,concelho,descricao_concelho,codigo_postal,medida,montante
0,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,13.0,PORTO,2.0,BAIAO,4640,III.6,"1228,13"
1,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,13.0,PORTO,2.0,BAIAO,4640,II.1,"1346,37"
2,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,13.0,PORTO,2.0,BAIAO,4640,IV/A.15,"8696,19"
3,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,13.0,PORTO,2.0,BAIAO,4640,IV/A.18,"3198,96"
5,2016,A A JOSÉ MANUEL SOARES DA ENCARNAÇÃO,8.0,FARO,2.0,ALCOUTIM,8970,IV/A.8,"5743,5"
7,2016,A A MARIA LUISA PALMA M RODRIGUES,8.0,FARO,16.0,VILA REAL DE SANTO ANTONIO,8900,IV/A.8,510


In [8]:
df.tail()

,exercicio,denominacao,distrito,descricao_distrito,concelho,descricao_concelho,codigo_postal,medida,montante
691080,2016,5 RIOS - ASSOCIAÇÃO AGRÍCOLA DO VALADO DOS FRADES,10.0,LEIRIA,11.0,NAZARE,2450,IV/A.2,"11617,76"
691082,2016,"7P, LDA",13.0,PORTO,12.0,PORTO,4200,III.6,"366,58"
691083,2016,"7P, LDA",13.0,PORTO,12.0,PORTO,4200,II.1,"203,44"
691084,2016,"7P, LDA",13.0,PORTO,12.0,PORTO,4200,IV/A.15,"2029,72"
691085,2016,"7P, LDA",13.0,PORTO,12.0,PORTO,4200,IV/A.18,"1923,85"


In [9]:
df = df.rename(columns={
        'exercicio': 'year',
        'medida': 'scheme',
        'montante': 'amount',
        'denominacao': 'recipient_name',
        'codigo_postal': 'recipient_postcode',
    })
df['distrito'] = pd.to_numeric(df['distrito'], errors='coerce')
df['concelho'] = pd.to_numeric(df['concelho'], errors='coerce')

distrito_dict = df.groupby(['distrito']).first()['descricao_distrito'].to_dict()
concelho_dict = df.groupby(['concelho']).first()['descricao_concelho'].to_dict()
df['distrito'] = df['distrito'].map(distrito_dict)
df['concelho'] = df['concelho'].map(concelho_dict)
df = df.drop(['descricao_distrito', 'descricao_concelho'], 1)
df.head()

,year,recipient_name,distrito,concelho,recipient_postcode,scheme,amount
0,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,III.6,"1228,13"
1,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,II.1,"1346,37"
2,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,IV/A.15,"8696,19"
3,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,IV/A.18,"3198,96"
5,2016,A A JOSÉ MANUEL SOARES DA ENCARNAÇÃO,FARO,BAIAO,8970,IV/A.8,"5743,5"


In [10]:
df['concelho'] = df['concelho'].fillna('')
df['distrito'] = df['distrito'].fillna('')
df.head()

,year,recipient_name,distrito,concelho,recipient_postcode,scheme,amount
0,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,III.6,"1228,13"
1,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,II.1,"1346,37"
2,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,IV/A.15,"8696,19"
3,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,IV/A.18,"3198,96"
5,2016,A A JOSÉ MANUEL SOARES DA ENCARNAÇÃO,FARO,BAIAO,8970,IV/A.8,"5743,5"


In [11]:
df['recipient_location'] = df.apply(lambda x: u'%s, %s' % (x['concelho'] or '', x['distrito']), 1)
df['amount'] = pd.to_numeric(df['amount'].str.replace(',', '.'))
df.head()

,year,recipient_name,distrito,concelho,recipient_postcode,scheme,amount,recipient_location
0,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,III.6,1228.13,"BAIAO, PORTO"
1,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,II.1,1346.37,"BAIAO, PORTO"
2,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,IV/A.15,8696.19,"BAIAO, PORTO"
3,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,IV/A.18,3198.96,"BAIAO, PORTO"
5,2016,A A JOSÉ MANUEL SOARES DA ENCARNAÇÃO,FARO,BAIAO,8970,IV/A.8,5743.50,"BAIAO, FARO"


In [12]:
len(df)

489927

In [13]:
from slugify import slugify
df['recipient_id'] = df.apply(lambda x: 'PT-%s-%s' % (x['recipient_postcode'], slugify(x['recipient_name'])), 1)
df = df.drop(['distrito', 'concelho'], 1)
df['currency'] = 'EUR'
df['country'] = 'PT'
df.head()

,year,recipient_name,recipient_postcode,scheme,amount,recipient_location,recipient_id,currency,country
0,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,4640,III.6,1228.13,"BAIAO, PORTO",PT-4640-a-d-servicos-e-investimentos-lda,EUR,PT
1,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,4640,II.1,1346.37,"BAIAO, PORTO",PT-4640-a-d-servicos-e-investimentos-lda,EUR,PT
2,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,4640,IV/A.15,8696.19,"BAIAO, PORTO",PT-4640-a-d-servicos-e-investimentos-lda,EUR,PT
3,2016,A & D SERVIÇOS E INVESTIMENTOS LDA,4640,IV/A.18,3198.96,"BAIAO, PORTO",PT-4640-a-d-servicos-e-investimentos-lda,EUR,PT
5,2016,A A JOSÉ MANUEL SOARES DA ENCARNAÇÃO,8970,IV/A.8,5743.50,"BAIAO, FARO",PT-8970-a-a-jose-manuel-soares-da-encarnacao,EUR,PT


In [14]:
df['year'].value_counts()

2016    489927
Name: year, dtype: int64

In [15]:
df.to_csv('pt_%s.csv.gz' % YEAR, index=False, encoding='utf-8', compression='gzip')